# Chains


**Chain interface**
```javascript
import { CallbackManagerForChainRun } from "langchain/callbacks";
import { BaseMemory } from "langchain/memory";
import { ChainValues } from "langchain/schema";

abstract class BaseChain {
  memory?: BaseMemory;

  /**
   * Run the core logic of this chain and return the output
   */
  abstract _call(
    values: ChainValues,
    runManager?: CallbackManagerForChainRun
  ): Promise<ChainValues>;

  /**
   * Return the string type key uniquely identifying this class of chain.
   */
  abstract _chainType(): string;

  /**
   * Return the list of input keys this chain expects to receive when called.
   */
  abstract get inputKeys(): string[];

  /**
   * Return the list of output keys this chain will produce when called.
   */
  abstract get outputKeys(): string[];
}
```

In [10]:
import { OpenAI } from "npm:langchain/llms/openai";
import { PromptTemplate } from "npm:langchain/prompts";
import { LLMChain } from "npm:langchain/chains";

// We can construct an LLMChain from a PromptTemplate and an LLM.
const model = new OpenAI({ temperature: 0 });
const prompt = PromptTemplate.fromTemplate(
  "What is a good name for a company that makes {product}?"
);

In [ ]:
const chain = new LLMChain({ llm: model, prompt });

// Since this LLMChain is a single-input, single-output chain, we can also `run` it.
// This convenience method takes in a string and returns the value
// of the output key field in the chain response. For LLMChains, this defaults to "text".
const res = await chain.run("colorful socks");
console.log( res );

An ```SimpleSequentialChain``` is a chain that allows you to join multiple single-input/single-output chains into one chain

In [12]:
import { SimpleSequentialChain, LLMChain } from "npm:langchain/chains";

const criticTemplate = `
You are a restaurant critic. You've tasted {dish} and are expressing your impression of it. 
The review can be either negative or positive.
`;

const assistantTemplate = `
You are an assistant bot. Your job is to make the customer feel heard and understood.
Reflect on the input you receive.

text: {review}
`;

const criticPromptTemplate = new PromptTemplate({
  template: criticTemplate,
  inputVariables: ["dish"],
});

const assistantPromptTemplate = new PromptTemplate({
  template: assistantTemplate,
  inputVariables: ["review"],
});


const reviewChain1 = new LLMChain({ llm: model, prompt: criticPromptTemplate });
const reviewChain2 = new LLMChain({ llm: model, prompt: assistantPromptTemplate });

const overallChain = new SimpleSequentialChain({
  chains: [reviewChain1, reviewChain2],
  verbose: false,
});

const result = await overallChain.run("Pizza Salami");

console.log(result);


Thank you for your feedback! It sounds like you really enjoyed the Pizza Salami. We're so glad to hear that the crust was crisp, the salami was flavorful and juicy, the cheese was melted to perfection, and the tomato sauce was just the right amount of tangy. We appreciate your recommendation and we hope you enjoy your next meal with us!


## LangChain Expression Language

# Interface

In an effort to make it as easy as possible to create custom chains, we've implemented a ["Runnable"](/docs/api/schema_runnable/classes/Runnable) protocol that most components implement.
This is a standard interface with a few different methods, which make it easy to define custom chains as well as making it possible to invoke them in a standard way. The standard interface exposed includes:

- `stream`: stream back chunks of the response
- `invoke`: call the chain on an input
- `batch`: call the chain on a list of inputs

The **input type** varies by component :

| Component      | Input Type                                          |
| -------------- | --------------------------------------------------- |
| Prompt         | Object                                              |
| Retriever      | Single string                                       |
| LLM, ChatModel | Single string, list of chat messages or PromptValue |
| Tool           | Single string, or object, depending on the tool     |
| OutputParser   | The output of an LLM or ChatModel                   |

The **output type** also varies by component :

| Component    | Output Type           |
| ------------ | --------------------- |
| LLM          | String                |
| ChatModel    | ChatMessage           |
| Prompt       | PromptValue           |
| Retriever    | List of documents     |
| Tool         | Depends on the tool   |
| OutputParser | Depends on the parser |

You can combine runnables (and runnable-like objects such as functions and objects whose values are all functions) into sequences in two ways:

- Call the `.pipe` instance method, which takes another runnable-like as an argument
- Use the `RunnableSequence.from([])` static method with an array of runnable-likes, which will run in sequence when invoked


## Examples

In [13]:
import { ChatOpenAI } from "npm:langchain/chat_models/openai";

const model = new ChatOpenAI({});
const promptTemplate = PromptTemplate.fromTemplate(
  "You are a restaurant critic. Express your impression about {dish} in 10 words"
);

In [ ]:
const chain = promptTemplate.pipe(model);

const stream = await chain.stream({ dish: "Pizza Salami" });

for await (const chunk of stream) {
  console.log(chunk?.content);
}

In [ ]:
import { RunnableSequence } from "npm:langchain/schema/runnable";

// You can also create a chain using an array of runnables
const chain = RunnableSequence.from([promptTemplate, model]);

const result = await chain.invoke({ dish: "Pizza Salami" });

console.log(result);

In [ ]:
const chain = promptTemplate.pipe(model);

const result = await chain.batch([{ dish: "Pizza" }, { dish: "Sushi" }]);

console.log(result);

## Back to LLMChain expamle

In [ ]:
import { PromptTemplate } from "npm:langchain/prompts";
import { StringOutputParser } from "npm:langchain/schema/output_parser";
import { RunnableSequence } from "npm:langchain/schema/runnable";

const criticPrompt = PromptTemplate.fromTemplate(
    `
    You are a restaurant critic. You've tasted {dish} and are expressing your impression of it. 
    The review can be either negative or positive.
    `
);

const assistantPrompt = PromptTemplate.fromTemplate(
  `
  You are an assistant bot. Your job is to make the customer feel heard and understood.
  Reflect on the input you receive.
  
  text: {review}
  `
);

const model = new ChatOpenAI({});

const criticChain = criticPrompt.pipe(model).pipe(new StringOutputParser());

const combinedChain = RunnableSequence.from([
  {
    review: criticChain,
  },
  assistantPrompt,
  model,
  new StringOutputParser(),
]);

const result = await combinedChain.invoke({
  dish: "Pizza"
});

console.log(result);

## Multi-model chains

In [ ]:
import { ChatOpenAI } from "npm:langchain/chat_models/openai";
import { PromptTemplate } from "npm:langchain/prompts";
import { StringOutputParser } from "npm:langchain/schema/output_parser";
import { RunnableSequence } from "npm:langchain/schema/runnable";
import { HfInference } from 'npm:@huggingface/inference';

const criticPrompt = PromptTemplate.fromTemplate(
    `
    You are a restaurant critic. You've tasted {dish} and are expressing your impression of it. 
    `
);

const assistantPrompt = PromptTemplate.fromTemplate(
  `
  Generate image of the {dishdescription}.
  `
);

const model = new ChatOpenAI({});
const hfi = new HfInference();

const criticChain = criticPrompt.pipe(model).pipe(new StringOutputParser());

const combinedChain = RunnableSequence.from([
  criticChain,
  (imagePrompt: string) => hfi.textToImage({
      model: 'stabilityai/stable-diffusion-2',
      inputs: imagePrompt,
      parameters: { negative_prompt: 'blurry' },
  })
]);

const imageBlob = await combinedChain.invoke({
  dish: "Pizza"
});

const imageBuf = await imageBlob.arrayBuffer();

const filePath = './test.jpg';
const bytes = new Uint8Array(imageBuf);
await Deno.writeFile(filePath, bytes);

console.log("![Generated Image](./test.jpg)");
